In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库
import time
import gc
import math

In [2]:
print('train')
train = pd.read_table('train.txt',sep=' ',index_col=False)

train


In [3]:
train.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')

In [4]:
#coding:utf-8
import pandas as pd
import time
import gc
from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['day'] = data['context_timestamp_tmp'].dt.day
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
    del data['shop_score_service']
    del data['shop_score_description']
    del data['shop_review_positive_rate']
    del data['shop_score_delivery']
    return data

In [5]:
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [6]:
test_a = pd.read_table('test.txt',sep=' ',index_col=False) 
test_a = pre_process(test_a)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [7]:
train.columns

Index(['instance_id', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_star_level', 'is_trade', 'category_0',
       'category_1', 'category_2', 'property_0', 'property_1', 'property_2',
       'day', 'halfhour'],
      dtype='object')

In [8]:
train.groupby('category_1').count()

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_review_num_level,shop_star_level,is_trade,category_0,category_2,property_0,property_1,property_2,day,halfhour
category_1,,,,,,,,,,,,,,,,,,,,,
1147074168968532252,73756,73756,73756,73756,73756,73756,73756,73756,73756,73756,...,73756,73756,73756,73756,73756,73756,73756,73756,73756,73756
1367177154073382718,33724,33724,33724,33724,33724,33724,33724,33724,33724,33724,...,33724,33724,33724,33724,33724,33724,33724,33724,33724,33724
1852600517265062354,491904,491904,491904,491904,491904,491904,491904,491904,491904,491904,...,491904,491904,491904,491904,491904,491904,491904,491904,491904,491904
1909641874861640857,848040,848040,848040,848040,848040,848040,848040,848040,848040,848040,...,848040,848040,848040,848040,848040,848040,848040,848040,848040,848040
1916390345133212703,252098,252098,252098,252098,252098,252098,252098,252098,252098,252098,...,252098,252098,252098,252098,252098,252098,252098,252098,252098,252098
1920084168104334820,14371,14371,14371,14371,14371,14371,14371,14371,14371,14371,...,14371,14371,14371,14371,14371,14371,14371,14371,14371,14371
2211060154630359130,367850,367850,367850,367850,367850,367850,367850,367850,367850,367850,...,367850,367850,367850,367850,367850,367850,367850,367850,367850,367850
2871729383671301763,763775,763775,763775,763775,763775,763775,763775,763775,763775,763775,...,763775,763775,763775,763775,763775,763775,763775,763775,763775,763775
3089254302947620489,51563,51563,51563,51563,51563,51563,51563,51563,51563,51563,...,51563,51563,51563,51563,51563,51563,51563,51563,51563,51563


In [9]:
train.append(test_a).groupby('category_1').count()

,category_0,category_2,context_id,context_page_id,context_timestamp,day,halfhour,instance_id,is_trade,item_brand_id,...,property_1,property_2,shop_id,shop_review_num_level,shop_star_level,user_age_level,user_gender_id,user_id,user_occupation_id,user_star_level
category_1,,,,,,,,,,,,,,,,,,,,,
1147074168968532252,76514,76514,76514,76514,76514,76514,76514,76514,73756,76514,...,76514,76514,76514,76514,76514,76514,76514,76514,76514,76514
1367177154073382718,34601,34601,34601,34601,34601,34601,34601,34601,33724,34601,...,34601,34601,34601,34601,34601,34601,34601,34601,34601,34601
1852600517265062354,515093,515093,515093,515093,515093,515093,515093,515093,491904,515093,...,515093,515093,515093,515093,515093,515093,515093,515093,515093,515093
1909641874861640857,895856,895856,895856,895856,895856,895856,895856,895856,848040,895856,...,895856,895856,895856,895856,895856,895856,895856,895856,895856,895856
1916390345133212703,264425,264425,264425,264425,264425,264425,264425,264425,252098,264425,...,264425,264425,264425,264425,264425,264425,264425,264425,264425,264425
1920084168104334820,14922,14922,14922,14922,14922,14922,14922,14922,14371,14922,...,14922,14922,14922,14922,14922,14922,14922,14922,14922,14922
2211060154630359130,386265,386265,386265,386265,386265,386265,386265,386265,367850,386265,...,386265,386265,386265,386265,386265,386265,386265,386265,386265,386265
2871729383671301763,803148,803148,803148,803148,803148,803148,803148,803148,763775,803148,...,803148,803148,803148,803148,803148,803148,803148,803148,803148,803148
3089254302947620489,53840,53840,53840,53840,53840,53840,53840,53840,51563,53840,...,53840,53840,53840,53840,53840,53840,53840,53840,53840,53840


In [10]:
temp = pd.DataFrame()

In [11]:
temp['category_1'] = (train[['category_1','is_trade']].groupby('category_1').sum()/train[['category_1','is_trade']].groupby('category_1').count()).reset_index()[['category_1']]

In [12]:
temp['c_pv'] = (train[['category_1','is_trade']].groupby('category_1').sum()/train[['category_1','is_trade']].groupby('category_1').count()).reset_index()[['is_trade']]

In [13]:
temp

,category_1,c_pv
0,1147074168968532252,0.011375
1,1367177154073382718,0.009637
2,1852600517265062354,0.008959
3,1909641874861640857,0.008218
4,1916390345133212703,0.017656
5,1920084168104334820,0.005706
6,2211060154630359130,0.021272
7,2871729383671301763,0.026496
8,3089254302947620489,0.004848
9,3348197449185791127,0.029382


In [14]:
train = pd.merge(train[['instance_id','category_1']],temp[['category_1','c_pv']],on=['category_1'],how='left')
test_a = pd.merge(test_a[['instance_id','category_1']],temp[['category_1','c_pv']],on=['category_1'],how='left')

In [15]:
train

,instance_id,category_1,c_pv
0,7118167540371,8769426218101861255,0.006800
1,43508800019098,6670526099037031245,0.012439
2,78067858668749,6693726201323251689,0.009208
3,137525720365644,6670526099037031245,0.012439
4,151193932520734,3613783563199627217,0.007395
5,191034067765499,1367177154073382718,0.009637
6,221786632908010,7314150500379498593,0.039434
7,233375766178961,5685690139879409547,0.007608
8,235542268576836,1909641874861640857,0.008218
9,271028956022181,6693726201323251689,0.009208


In [16]:
train[['instance_id','c_pv']].to_csv('train_pv.csv',index=False) 
test_a[['instance_id','c_pv']].to_csv('test_pv.csv',index=False) 